In [1]:
root_dir = '/home2/glee/dissertation/1_tech_gen_impact/class2class/Tech_Gen/'
master_dir = '/home2/glee/dissertation/1_tech_gen_impact/master/Tech_Gen/'
import sys
sys.path.append(root_dir)

import copy
import gc
import os
import argparse
import math
import time
import pickle
import re
import multiprocess as mp
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
sys.path.append("/share/tml_package")
from tml import utils
from scipy import io
from tqdm import tqdm
from collections import OrderedDict

import torch
from torch.nn import functional as F
from torch.nn import DataParallel as DP
from torch.utils.data import TensorDataset, DataLoader, Subset, Dataset
from accelerate import Accelerator
import pytorch_model_summary

import optuna
from optuna.samplers import RandomSampler, TPESampler
from optuna.integration import SkoptSampler

import numpy as np
import pandas as pd
import scipy.stats
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import matthews_corrcoef, precision_recall_fscore_support, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

from data import TechDataset, CVSampler
from models import Transformer, Predictor
from train_utils import EarlyStopping, perf_eval, objective_cv, build_model, train_model, validate_model_mp
from utils import token2class, DotDict, to_device

from cleantext.sklearn import CleanTransformer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# PART 1: Configuration

In [14]:
analysis_date = "2023-04-28_1241"
args = argparse.Namespace(
#     data_type="class+claim",
#     data_file=None,
#     target_ipc=None,
#     pred_type="classification",
#     n_TC = 5,
#     use_pretrained_tokenizer=False,
#     do_train=None,
    do_eval = True,
#     do_tune=None,
#     n_folds=None,
#     batch_size=512,
#     max_epochs=20,
#     use_accelerator=None,
    do_save=False,
#     n_gpus=4,
#     light=True,
    config_file=os.path.join(root_dir, "configs", "USED_configs", "[CONFIGS]"+analysis_date+".json"),
#     config_file=None,
    eval_train_set=False)

project_data_dir = os.path.join(master_dir, "data")
data_dir = os.path.join("/home2/glee/patent_data/data/")
model_dir = os.path.join(root_dir, "models")
result_dir = os.path.join(root_dir, "results")
config_dir = os.path.join(root_dir, "configs")

## parse configuration file
# args = parser.parse_args()
if args.config_file is not None:
    config_file = args.config_file
else:
    config_file = os.path.join(config_dir, "configs_light.json") if args.light else os.path.join(config_dir, "configs.json")
if args.do_eval: args.do_train = False
configs = DotDict().load(config_file)
org_config_keys = {key: list(configs[key].keys()) for key in configs.keys()}

# parse command line arguments
instant_configs = {key: value for (key, value) in vars(args).items() if value is not None} # if any argument passed when main.py executed
instant_configs_for_update = {configkey: {key: value for (key,value) in instant_configs.items() if key in org_config_keys[configkey]} for configkey in org_config_keys.keys()}
for key, value in configs.items():
    value.update(instant_configs_for_update[key])

## assign loss weights
if configs.model.model_type == "enc-pred-dec":
    configs.train.loss_weights["recon"] = configs.train.loss_weights["recon"] / sum(configs.train.loss_weights.values())
    configs.train.loss_weights["y"] = 1 - configs.train.loss_weights["recon"]
elif configs.model.model_type == "enc-pred":
    configs.train.loss_weights = {"recon": 0, "y": 1}
elif configs.model.model_type == "enc-dec":
    configs.train.loss_weights = {"recon": 1, "y": 0}

## assign devices
if configs.train.use_accelerator:
    accelerator = Accelerator()
    device_ids = list(range(torch.cuda.device_count()))
    device = accelerator.device
    configs.train.update({"accelerator": accelerator})
else:
    if torch.cuda.is_available():
        device_ids = list(range(torch.cuda.device_count()))
        gpu_usages = [np.sum([float(usage.split("uses")[-1].replace(" ","").replace("MB","")) for usage in torch.cuda.list_gpu_processes(id).split("GPU memory") if not usage=="" and "no processes are running" not in usage]) for id in device_ids]
        device_ids = np.argsort(gpu_usages)[:configs.train.n_gpus]
        device_ids = list(map(lambda x: torch.device('cuda', x),list(device_ids)))
        device = device_ids[0] # main device
        torch.cuda.set_device(device)
    else:
        device = torch.device('cpu')
        device_ids = []

## extract configurations for dataset
config_period = "["+"-".join([str(year) for year in configs.data.target_period])+"]"
config_ipcs = str(configs.data.target_ipc).replace("\'","").replace(" ","")
config_keywords = str(configs.data.target_keywords).replace("\'","").replace(" ","")

## update configurations
configs.data.update({"root_dir": root_dir,
                        "data_dir": data_dir,
                        "model_dir": model_dir,
                        "result_dir": result_dir,
                        "pretrained_enc": configs.model.pretrained_enc,
                        "pretrained_dec": configs.model.pretrained_dec,
                        "data_nrows": None,
                        "data_file": "collection_" + "".join([config_keywords, config_ipcs, config_period]) + ".csv"})
configs.train.update({"device": device,
                        "device_ids": device_ids,
                        "root_dir": root_dir,
                        "data_dir": data_dir,
                        "model_dir": model_dir,
                        "use_keywords": configs.data.use_keywords,
                        "early_stop_patience": int(0.3*configs.train.max_epochs)})
configs.model.update({"device": device,
                        "device_ids": device_ids,
                        "n_directions": 2 if configs.model.bidirec else 1,
                        "use_accelerator": configs.train.use_accelerator})

## Set hyperparameters for model training (To be TUNED)
if configs.train.do_train and configs.train.do_tune:
    n_layers = configs.model.n_layers = None
    d_embedding = configs.model.d_embedding = None
    d_enc_hidden = configs.model.d_enc_hidden = None
    d_pred_hidden = configs.model.d_pred_hidden = None
    learning_rate = configs.train.learning_rate = None
    batch_size = configs.train.batch_size = None
    config_name = "HPARAM_TUNING"
    final_model_path = None
else:
    n_layers = configs.model.n_layers
    d_embedding = configs.model.d_embedding
    d_enc_hidden = configs.model.d_enc_hidden
    d_pred_hidden = configs.model.d_pred_hidden
    d_latent = configs.model.d_latent

    ## set filename for model
    key_components = {"data": ["ipc_level", "max_seq_len_class", "max_seq_len_claim", "vocab_size"], "model": ["n_layers", "d_hidden", "d_pred_hidden", "d_latent", "d_embedding", "d_ff", "n_head", "d_head"], "train": ["learning_rate", "batch_size", "max_epochs"]}
    model_config_name = "".join([config_keywords, config_ipcs, config_period]) + "data"
    for key in ["model", "train"]:
        for component in key_components[key]:
            model_config_name += f"[{str(configs[key][component])}]{component}"
    final_model_path = os.path.join(model_dir, f"[MODEL]{model_config_name}.ckpt")

configs.train.update({"model_config_name": model_config_name, "final_model_path": final_model_path})


# PART 2: Dataset setting

In [93]:
''' PART 2: Dataset setting '''
tstart = time.time()
dataset_config_name = "".join([config_keywords, config_ipcs, config_period]) + "data"
for component in key_components["data"]:
    dataset_config_name += f"[{str(configs.data[component])}]{component}"
dataset_path = os.path.join(project_data_dir, "pickled_dataset", "[DATASET]"+dataset_config_name+".pickle")

if os.path.exists(dataset_path) and args.do_save is False:
    print("Load pickled dataset...")
    with open(dataset_path, "rb") as f:
        tech_dataset = pickle.load(f)   # Load pickled dataset if dataset with same configuration already saved
        if tech_dataset.pretrained_enc != configs.data.pretrained_enc or tech_dataset.pretrained_dec != configs.data.pretrained_dec:
            tech_dataset.pretrained_enc = configs.data.pretrained_enc
            tech_dataset.pretrained_dec = configs.data.pretrained_dec
            tech_dataset.tokenizers = tech_dataset.get_tokenizers()
        for tk in tech_dataset.tokenizers.values():
            if "vocab_size" not in dir(tk):
                tk.vocab_size = tk.get_vocab_size()
        tech_dataset.use_keywords = configs.data.use_keywords
        ## load saved rawdata
        if tech_dataset.rawdata is None:
            tech_dataset.rawdata = pd.read_csv(os.path.join(data_dir, configs.data.data_file), low_memory=False)
    print("Pickled dataset loaded")
else:
    print("Make dataset...")
    if args.debug:
        configs.data.update({"data_nrows": 1000})
        dataset_path += ".debug"
    tech_dataset = TechDataset(configs.data)
    if not args.debug:
        rawdata_for_save = copy.deepcopy(tech_dataset.rawdata)
        with open(dataset_path, "wb") as f:
            tech_dataset.rawdata = None
            pickle.dump(tech_dataset, f)
        tech_dataset.rawdata = rawdata_for_save
tend = time.time()
print(f"{np.round(tend-tstart,4)} sec elapsed for loading patents for class [{configs.data.target_ipc}]")

configs.model.update({"tokenizers": tech_dataset.tokenizers,
                    "n_enc_seq_claim": tech_dataset.max_seq_len_claim,
                    "n_dec_seq_claim": tech_dataset.max_seq_len_claim,
                    "n_enc_seq_class": tech_dataset.max_seq_len_class,
                    "n_dec_seq_class": tech_dataset.max_seq_len_class,
                    "n_outputs": 1 if configs.data.pred_type=="regression" else tech_dataset.n_outputs,
                    "i_padding": tech_dataset.tokenizers["class_enc"].pad_id})

Load pickled dataset...
Pickled dataset loaded
10.859 sec elapsed for loading patents for class [['H01L', 'H10']]


## Inference

In [94]:
final_model_path

'/home2/glee/dissertation/1_tech_gen_impact/class2class/Tech_Gen/models/[MODEL][semiconductor,silicon,chip][H01L,H10][2007-2012]data[4]n_layers[64]d_hidden[8]d_pred_hidden[64]d_latent[256]d_embedding[16]d_ff[4]n_head[32]d_head[0.0005]learning_rate[256]batch_size[100]max_epochs.ckpt'

In [26]:
final_model = build_model(configs.model, tokenizers=tech_dataset.tokenizers)
if os.path.exists(final_model_path):
    best_states = torch.load(final_model_path)
else:
    raise Exception("Model need to be trained first")
converted_states = OrderedDict()
for k, v in best_states.items():
    if 'module' not in k:
        k = 'module.'+k
    else:
        k = k.replace('features.module.', 'module.features.')
    converted_states[k] = v
final_model.load_state_dict(converted_states)

del best_states
del converted_states
torch.cuda.empty_cache()
print("Model successfully loaded")

Model successfully loaded


In [ ]:
# Copy predictor
temp_path = os.path.join(model_dir, "temp", "temp.ckpt")
predictor = Predictor(final_model.module.config).to(final_model.module.device)
torch.save(final_model.module.predictor.state_dict(), temp_path)
predictor.load_state_dict(torch.load(temp_path, map_location=final_model.module.device))

In [95]:
result_path = os.path.join(root_dir, "results")

used_train_data = pd.read_excel(os.path.join(result_path, "[DATASET]"+analysis_date+".xlsx"), sheet_name="TRAIN_dataset")
used_test_data = pd.read_excel(os.path.join(result_path, "[DATASET]"+analysis_date+".xlsx"), sheet_name="TEST_dataset")
used_train_index = tech_dataset.data.index.get_indexer(pd.Index(used_train_data["number"]))
used_test_index = tech_dataset.data.index.get_indexer(pd.Index(used_test_data["number"]))

In [29]:
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt

In [30]:
batch_size = 16

In [96]:
used_train_dataset = Subset(tech_dataset, used_train_index)
train_loader = DataLoader(used_train_dataset, batch_size=batch_size, drop_last=True)

In [34]:
zs, ys, preds = [], [], []
newzs = []
for batch_data in tqdm(train_loader):
    batch_data = to_device(batch_data, final_model.module.device)
    y = batch_data["targets"].cpu().detach().numpy()
    
    enc_outputs, z, mu, logvar = final_model.module.encode(batch_data["text_inputs"])
    pred_outputs = final_model.module.predictor(z)
    torch.cuda.empty_cache()

    zs.append(z.cpu().detach().numpy())
    ys.append(y)
    preds.append(pred_outputs.argmax(1).cpu().detach().numpy())
    
    torch.cuda.empty_cache()
                                                
zs = np.concatenate(zs)
ys = np.concatenate(ys)
preds = np.concatenate(preds)

100%|██████████| 2073/2073 [01:15<00:00, 27.36it/s]


In [ ]:
import datetime
from nltk.translate.bleu_score import sentence_bleu
col_years = ["<1976"] + np.arange(1976,2023).astype(str).tolist()
latest_year = datetime.datetime.now().year - 1
n_TC = configs.data.n_TC

visualize = False

In [ ]:
ref_config_period = "[2012-2017]"
ref_data_file = "collection_" + "".join([config_keywords, config_ipcs, ref_config_period]) + ".csv"
ref_configs = copy.deepcopy(configs)
ref_configs.data.update({"target_period": ref_config_period, "data_file": ref_data_file})
ref_dataset = TechDataset(ref_configs.data)

In [283]:
used_rawdata = tech_dataset.rawdata.set_index("number")
total_data = pd.concat([tech_dataset.data, ref_dataset.data], axis=0)

In [795]:
cnt_nonexist = 0
cnt_noFC = 0
for idx in tqdm(range(len(used_test_index))):
    if used_test_data.iloc[idx]["TC5"] > 0:
        forward_refs = used_rawdata.loc[used_test_data.iloc[idx]["number"]]["forward_refs"].split(";")
        ref_info = total_data.loc[[ref for ref in forward_refs if ref in total_data.index]]
        if len(ref_info) == 0:
            cnt_nonexist += 1
    else:
        cnt_noFC += 1

100%|██████████| 3687/3687 [00:08<00:00, 426.22it/s]


In [802]:
cnt_nonexist / 3687

0.15975047464062925

In [803]:
cnt_noFC / 3687

0.3232980743151614

In [799]:
cnt_nonexist+cnt_noFC

1781

In [800]:
3687 - 1781

1906

In [801]:
1906 - 1334

572

In [879]:
def validate_reliability(idx=None):
    cnt_nonexist = 0
    cnt_noFC = 0
    cnt_diverge = 0
    input_class = torch.tensor(tech_dataset.tokenizers["class_enc"].encode(tech_dataset.X_class[used_test_index][idx])).unsqueeze(0)
    input_claim = tech_dataset.tokenize(tech_dataset.tokenizers["claim_enc"], tech_dataset.X_claim[used_test_index][idx])
    input_claim = {k: v.unsqueeze(0) for k, v in input_claim.items()}
    batch_input = {"class": torch.tensor(input_class), "claim": input_claim}
    input_inf = to_device(batch_input, final_model.module.device)

    output_class = torch.tensor(tech_dataset.tokenizers["class_dec"].encode(tech_dataset.X_class[used_test_index][idx])).unsqueeze(0)
    batch_output = {"text_outputs": torch.tensor(output_class)}
    output_inf = to_device(batch_output, final_model.module.device)

    near_mean_idx = np.argsort(np.sum(abs(zs - np.mean(zs, axis=0)), axis=1))[:2500]
    near_mean_idx_ = np.union1d(near_mean_idx, np.random.choice(np.where(ys==1)[0], 50))
    enc_outputs, z, mu, logvar = final_model.module.encode(input_inf)
    org_z = copy.deepcopy(z.view(1,-1).cpu().detach().numpy())
    pred_outputs = final_model.module.predict(z)
    org_y = copy.deepcopy(pred_outputs.argmax(1).cpu().detach().numpy())
    dec_inputs = None

    if used_test_data.iloc[idx]["TC5"] > 0:
        forward_refs = used_rawdata.loc[used_test_data.iloc[idx]["number"]]["forward_refs"].split(";")
        ref_info = total_data.loc[[ref for ref in forward_refs if ref in total_data.index]]
        if len(ref_info) == 0:
            pass
            print("Dataset does not have information about citing patents")
            cnt_nonexist += 1
            return (cnt_nonexist, cnt_noFC, cnt_diverge), None
        else:
            ref_ipcs = ref_info["ipcs"]
            ref_FCs = ref_info["TC"+str(n_TC)]

            if visualize:
                zs_for_tsne = np.concatenate([zs[near_mean_idx_], org_z])
                ys_for_tsne = np.concatenate([ys[near_mean_idx_], org_y])
                tsne = TSNE(early_exaggeration=10, learning_rate="auto", n_iter=500, init="random", verbose=0, metric="cosine", square_distances=True)
                z_tsne = tsne.fit_transform(zs_for_tsne)
                plt.scatter(z_tsne[:-1,0], z_tsne[:-1,1], c=ys_for_tsne[:-1], cmap="bwr")
                plt.scatter(z_tsne[-1,0], z_tsne[-1,1], c="k", marker="X")
                plt.text(z_tsne[-1,0]+0.5, z_tsne[-1,1]+0.5, "origin", weight="bold")
                plt.show()

            tokenizer = tech_dataset.tokenizers["class_dec"]

#             print(f"<< Iteration {0} >>")

#             org_text = tokenizer.decode_batch(input_class.cpu().detach().numpy())[0]
#             org_text = org_text[org_text.index(tokenizer.sos_token)+1:org_text.index(tokenizer.eos_token)]

#             dec_outputs = final_model.module.decode(z, enc_outputs, dec_inputs=None)
#             dec_outputs = dec_outputs.argmax(-1)

#             gen_text = tokenizer.decode_batch(dec_outputs.cpu().detach().numpy())[0]
#             gen_text = gen_text[gen_text.index(tokenizer.sos_token)+1:gen_text.index(tokenizer.eos_token)]
#             gen_text = [gen_text[0]] + list(np.array(gen_text[1:])[np.unique(gen_text[1:], return_index=True)[1]])
#             print("Original class:\n",org_text,"\nGenerated class:\n", gen_text,"\n")

            n_iter = 100
            step_size = 20

            optimised = False
            for i in range(n_iter):
#                 print(f"<< Iteration {i+1} >>")
                pred_outputs = final_model.module.predict(z)
                z.retain_grad()
                FC_estimated = np.round(np.exp(pred_outputs[0,1].item()), 4) # estimated forward citations
                
                FC_estimated_inv = np.round(np.exp(pred_outputs[0,0].item()), 4)
#                 if i % 1 == 0:
#                     print(f"Estimated prob. for L1 (L2): {FC_estimated} ({FC_estimated_inv})")
                pred_outputs[0,1].backward(retain_graph=True)

                grad_for_update = (step_size * z.grad)
                z_ = z + grad_for_update

                if visualize:
                    curr_z = copy.deepcopy(z_.view(1,-1).cpu().detach().numpy())
                    curr_y = copy.deepcopy(pred_outputs.argmax(1).cpu().detach().numpy())

                    zs_for_tsne = np.concatenate([zs[near_mean_idx_], org_z, curr_z])
                    ys_for_tsne = np.concatenate([ys[near_mean_idx_], org_y, curr_y])

                    z_tsne = tsne.fit_transform(zs_for_tsne)
                    plt.scatter(z_tsne[:-1,0], z_tsne[:-1,1], c=ys_for_tsne[:-1], cmap="bwr")
                    plt.scatter(z_tsne[-2,0], z_tsne[-2,1], c="k", marker="X")
                    plt.text(z_tsne[-2,0]+0.5, z_tsne[-2,1]+0.5, "origin", weight="bold")
                    plt.scatter(z_tsne[-1,0], z_tsne[-1,1], c="c", marker="x")
                    plt.text(z_tsne[-1,0]-4, z_tsne[-1,1]-2.5, f"Iter_{i}", c="c")
                    plt.show()

                z.grad.zero_()
                dec_outputs = final_model.module.decode(z_, enc_outputs, dec_inputs=None)
                dec_outputs = dec_outputs.argmax(-1)

                tokenizer = tech_dataset.tokenizers["class_dec"]
                gen_text = tokenizer.decode_batch(dec_outputs.cpu().detach().numpy())[0]
                gen_text = gen_text[gen_text.index(tokenizer.sos_token)+1:gen_text.index(tokenizer.eos_token)]
                if gen_text != []:
                    gen_text = [gen_text[0]] + list(np.array(gen_text[1:])[np.unique(gen_text[1:], return_index=True)[1]])                
                else: continue
                
#                 if not np.array_equal(org_text, gen_text) and FC_estimated>=0.5:
                if set(org_text)!=set(gen_text) and FC_estimated>=0.5:
                    optimised = True
#                     print(f"<< Iteration {i+1} >>")
#                     print(f"Estimated prob. for L1 (L2): {FC_estimated} ({FC_estimated_inv})")
                    print("Original class:\n",org_text,"\nGenerated class:\n", gen_text)
        #             print(f"REFS: {ref_ipcs}")
#                     temp_gen_text = np.concatenate([[t[0],t[:3],t[:4],t] for t in gen_text])
#                     temp_ref_ipcs = [np.concatenate([[t[0],t[:3],t[:4],t] for t in ipc]) for ipc in ref_ipcs]
#                     BLEU_scores = np.array([sentence_bleu([ipc], temp_gen_text, weights=(1.0,)) for ipc in temp_ref_ipcs])
#                     Jaccard_scores = np.array([len(set(ipc).intersection(set(temp_gen_text))) / len(set(ipc).union(set(temp_gen_text))) for ipc in temp_ref_ipcs])
        #             print(f"BLEU_scores: {np.round(BLEU_scores,4)}\n")
    
                    gen_text_breakdown = breakdown(gen_text)
                    ref_ipcs_breakdown = (ref_ipcs.apply(lambda x: breakdown(x)[0]), ref_ipcs.apply(lambda x: breakdown(x)[1]), ref_ipcs.apply(lambda x: breakdown(x)[2]), ref_ipcs)
    
                    for i in range(4):
                        temp_gen_text = gen_text_breakdown(i)
                        temp_ref_ipcs = ref_ipcs_breakdown(i)
                    
                        hit_index = temp_ref_ipcs.apply(lambda x: 1 if set(x)==set(temp_gen_text) else 0)==1
                        similar_refs = temp_ref_ipcs[hit_index].index
                        unsimilar_refs = temp_ref_ipcs[~hit_index].index
    #                     similar_refs = np.where(BLEU_scores>=score_threshold)[0]
    #                     unsimilar_refs = np.setdiff1d(np.arange(len(BLEU_scores)), similar_refs)
                        if len(similar_refs) == 0:
                            inclusion = 0
    #                         higher_impact = 0
                            higher_impact = None
                            print(f"{ref_ipcs}")
                            print(BLEU_scores)
                            print(Jaccard_scores)
                            print("Generated IPCs are not in citing patents' IPCs")
                        elif len(unsimilar_refs) == 0:
                            inclusion = 1
                            similar_mean_FC = np.mean(ref_FCs.loc[similar_refs])
                            if similar_mean_FC <= 0:
                                higher_impact = 0
                            else:
                                higher_impact = 1
                            print(f"similar patents' FCs: {np.round(similar_mean_FC, 4)}, no unsimilar patent")
                        else:
                            inclusion = 1
                            similar_mean_FC = np.mean(ref_FCs.loc[similar_refs])
                            unsimilar_mean_FC = np.mean(ref_FCs.loc[unsimilar_refs])
                            if similar_mean_FC >= unsimilar_mean_FC:
                                if similar_mean_FC <= 0:
                                    higher_impact = None
                                else:
                                    higher_impact = 1
                            else:
                                 higher_impact = 0   
    #                         higher_impact = 1 if similar_mean_FC >= unsimilar_mean_FC else 0
                            print(f"similar patents' FCs: {np.round(similar_mean_FC, 4)}, unsimilar patents' FCs: {np.round(unsimilar_mean_FC, 4)}")
                        if higher_impact==1:
                            print(f"Generated IPCs have higher impact")
                        print("\n")
                        break
                    
                z = z_
            if optimised:
                return (cnt_nonexist, cnt_noFC, cnt_diverge), {"index": idx, "patent_id": used_test_data.iloc[idx]["number"], "inclusion": inclusion, "higher_impact": higher_impact, "FC_estimated": FC_estimated}
#                 return gen_text, ref_ipcs, {"index": idx, "patent_id": used_test_data.iloc[idx]["number"], "inclusion": inclusion, "higher_impact": higher_impact, "FC_estimated": FC_estimated}
            else:
                cnt_diverge += 1
                return (cnt_nonexist, cnt_noFC, cnt_diverge), None
    else:
        pass
        print("Input patent does not have forward citation")
        cnt_noFC += 1
        return (cnt_nonexist, cnt_noFC, cnt_diverge), None

In [856]:
score_threshold=0.2

In [857]:
np.union1d(np.where(BLEU_scores>=score_threshold)[0], np.where(Jaccard_scores>=score_threshold)[0])

array([ 0,  1,  2,  6,  8,  9, 11, 12, 14, 15, 16, 23, 24, 27, 28, 29, 30,
       32, 33, 34, 35, 38, 42, 43, 44, 45, 47, 49, 51, 54, 55, 56, 57, 58,
       59, 60, 61, 63, 64, 65, 66, 67])

In [847]:
ref_ipcs[ref_ipcs.apply(lambda x: 1 if set(x)==set(["G06F17", "G06F09"]) else 0)==1].index

Index(['8448102', '8701071', '8769453', '8769453'], dtype='object', name='number')

In [848]:
forward_refs = used_rawdata.loc[used_test_data.iloc[67]["number"]]["forward_refs"].split(";")
ref_info = total_data.loc[[ref for ref in forward_refs if ref in total_data.index]]

In [849]:
ref_info

,number,main_ipc,sub_ipc,ipcs,claims,TC5,TC5_digitized,class
number,,,,,,,,
9779200,9779200,H01L23,"[G06F17, H01L21, H01L23]","[H01L23, G06F17, H01L21, H01L23]","1. A semiconductor chip, comprising: a first c...",3,0,113
9779200,9779200,H01L23,"[G06F17, H01L21, H01L23]","[H01L23, G06F17, H01L21, H01L23]","1. A semiconductor chip, comprising: a first c...",3,0,113
9741719,9741719,H01L21,"[G06F17, H01L21, H01L23, H01L27]","[H01L21, G06F17, H01L21, H01L23, H01L27]","1. An integrated circuit, comprising: a first ...",3,0,113
9281371,9281371,H01L29,"[H01L21, H01L23]","[H01L29, H01L21, H01L23]","1. A semiconductor device, comprising: a first...",5,0,113
8283701,8283701,H01L27,"[H01L29, H01L31]","[H01L27, H01L29, H01L31]","1. An integrated circuit device, comprising: a...",92,1,100
...,...,...,...,...,...,...,...,...
9009641,9009641,G06F17,"[H01L21, H01L27]","[G06F17, H01L21, H01L27]","1. A semiconductor device, comprising: a subst...",19,1,106
8872283,8872283,H01L27,"[G06F17, H01L27]","[H01L27, G06F17, H01L27]","1. An integrated circuit, comprising: a gate e...",4,0,113
8356268,8356268,G06F17,[H01L23],"[G06F17, H01L23]","1. An integrated circuit device, comprising: a...",104,1,106


In [850]:
ref_ipcs = ref_info["ipcs"]
ref_FCs = ref_info["TC"+str(n_TC)]


In [828]:
gen_text

['G06F17', 'G06F17', 'G06F19']

In [891]:
def breakdown(ipcs):
    return ([ipc[0] for ipc in ipcs], 
            [ipc[:3] for ipc in ipcs],
            [ipc[:4] for ipc in ipcs],
            ipcs)

In [892]:
breakdown(gen_text)

(['G', 'G', 'G'],
 ['G06', 'G06', 'G06'],
 ['G06F', 'G06F', 'G06F'],
 ['G06F17', 'G06F17', 'G06F19'])

In [903]:
(ref_ipcs.apply(lambda x: breakdown(x)[0]), ref_ipcs.apply(lambda x: breakdown(x)[1]), ref_ipcs.apply(lambda x: breakdown(x)[2]), ref_ipcs)[1]

number
9779200         [H01, G06, H01, H01]
9779200         [H01, G06, H01, H01]
9741719    [H01, G06, H01, H01, H01]
9281371              [H01, H01, H01]
8283701              [H01, H01, H01]
                     ...            
9009641              [G06, H01, H01]
8872283              [H01, G06, H01]
8356268                   [G06, H01]
8769453                   [G06, G06]
8769453                   [G06, G06]
Name: ipcs, Length: 68, dtype: object

In [809]:
temp_gen_text

array(['G', '06', 'F', '17', 'G', '06', 'F', '17', 'G', '06', 'F', '19'],
      dtype='<U2')

In [823]:
temp_gen_text = np.concatenate([[t[0],t[:3],t[:4],t] for t in gen_text[1:]])
temp_ref_ipcs = [np.concatenate([[t[0],t[:3],t[:4],t] for t in ipc]) for ipc in ref_ipcs]

In [824]:
temp_gen_text

array(['G', 'G06', 'G06F', 'G06F17', 'G', 'G06', 'G06F', 'G06F19'],
      dtype='<U6')

In [825]:
temp_ref_ipcs[-1]

array(['G', 'G06', 'G06F', 'G06F17', 'G', 'G06', 'G06F', 'G06F09'],
      dtype='<U6')

In [826]:
sentence_bleu([temp_ref_ipcs[-1]], temp_gen_text, weights=(1.0,))

0.875

In [827]:
len(set(temp_gen_text).intersection(set(temp_ref_ipcs[-1]))) / len(set(temp_gen_text).union(set(temp_ref_ipcs[-1])))

0.6666666666666666

In [774]:
np.concatenate([[t[0],t[:3],t[:4],t] for t in gen_text])

array(['G', 'G06', 'G06F', 'G06F17', 'G', 'G06', 'G06F', 'G06F17', 'G',
       'G06', 'G06F', 'G06F19'], dtype='<U6')

In [771]:
temp_gen_text

array(['G', '06', 'F', '17', 'G', '06', 'F', '17', 'G', '06', 'F', '19'],
      dtype='<U2')

In [876]:
res = validate_reliability(idx)

Dataset does not have information about citing patents


In [877]:
res

In [882]:
cnt_nonexist, cnt_noFC, cnt_diverge = 0, 0, 0
score_threshold = 0.6
dict_out = {"index": [], "patent_id": [], "inclusion": [], "higher_impact": [], "FC_estimated": []}
for idx in tqdm(range(len(used_test_index))):
# for idx in tqdm(range(100)):
    cnts, results = validate_reliability(idx)
    cnt_nonexist += cnts[0]
    cnt_noFC += cnts[1]
    cnt_diverge += cnts[2]
    if results is not None:
        for k,v in results.items():
            if v is not None:
                dict_out[k].append(v)

  0%|          | 4/3687 [00:00<09:32,  6.43it/s]

Original class:
 ['H01L29', 'H01L21'] 
Generated class:
 ['H01L29', 'A61B17']
number
9373573            [H01L21, H01L21, H01L23]
9847309            [H01L21, H01L21, H01L23]
9159665    [H01L21, H01L21, H01L23, H05K01]
8138079                    [H01L21, H01L21]
8138079                    [H01L21, H01L21]
                         ...               
8884443            [H01L21, H01L21, H01L23]
8823167                    [H01L23, H01L23]
8598691                    [H01L23, H01L29]
8841766                    [H01L23, H01L23]
9142527                    [H01L21, H01L23]
Name: ipcs, Length: 68, dtype: object
[0.227449   0.227449   0.13795479 0.375      0.375      0.30326533
 0.18393972 0.227449   0.227449   0.375      0.18393972 0.375
 0.375      0.375      0.227449   0.375      0.30326533 0.13795479
 0.13795479 0.227449   0.18393972 0.227449   0.227449   0.227449
 0.30326533 0.30326533 0.375      0.375      0.13795479 0.30326533
 0.30326533 0.04598493 0.13795479 0.13795479 0.5        0.3032653

  0%|          | 8/3687 [00:04<34:46,  1.76it/s]

Input patent does not have forward citation
Input patent does not have forward citation
Original class:
 ['H01L29', 'H01L21'] 
Generated class:
 ['H01L21', 'H01L21']
number
9640765    [H01L51, B82Y10, B82Y40, H01L29, H01L51]
9406888    [H01L21, B82Y10, B82Y40, H01L21, H01L51]
9559211                    [H01L29, H01L21, H01L29]
Name: ipcs, dtype: object
[0.16734762 0.22313016 0.53071433]
[0.27272727 0.4        0.8       ]
Generated IPCs are not in citing patents' IPCs


Dataset does not have information about citing patents
Input patent does not have forward citation


  0%|          | 11/3687 [00:04<23:09,  2.65it/s]

...

Original class:
 ['H01L29', 'H01L21'] 
Generated class:
 ['H01L21', 'H01L21']
number
7518151    [H01L27, H01L31, H01L33]
Name: ipcs, dtype: object
[0.45489799]
[0.42857143]
Generated IPCs are not in citing patents' IPCs


Original class:
 ['H01L29', 'H01L21'] 
Generated class:
 ['H01L21', 'B44C01']
number
9604339            [B24B37, B24B37]
9233452    [B24B37, B24B37, H01L21]
9233452    [B24B37, B24B37, H01L21]
9011207    [B24B37, B24B37, H01L21]
9199354    [B24B37, B24B37, H01L21]
Name: ipcs, dtype: object
[0.125      0.37908166 0.37908166 0.37908166 0.37908166]
[0.09090909 0.45454545 0.45454545 0.45454545 0.45454545]
Generated IPCs are not in citing patents' IPCs




100%|█████████▉| 3681/3687 [36:20<00:05,  1.05it/s]

Original class:
 ['H01L29', 'H01L21'] 
Generated class:
 ['H01L23', 'H01L23', 'H01L29']
number
9559005    [H01L21, H01L21, H01L23]
Name: ipcs, dtype: object
[0.83333333]
[0.66666667]
Generated IPCs are not in citing patents' IPCs


Input patent does not have forward citation


100%|██████████| 3687/3687 [36:24<00:00,  1.69it/s]

Dataset does not have information about citing patents
Original class:
 ['H01L29', 'H01L21'] 
Generated class:
 ['H01L23', 'H01L23', 'H01L29', 'H01L31']
number
9379201    [H01L21, H01L21, H01L23, H01L27, H01L29]
9105701                    [H01L23, H01L21, H01L23]
8810010                    [H01L29, H01L21, H01L23]
8928127                            [H01L23, H01L23]
9224702                            [H01L21, H01L23]
9093462    [H01L23, H01L23, H01L25, H01L27, H01L29]
Name: ipcs, dtype: object
[0.68145069 0.6875     0.6875     0.5        0.4375     0.73012573]
[0.625      0.57142857 0.71428571 0.66666667 0.57142857 0.625     ]
Generated IPCs are not in citing patents' IPCs


Input patent does not have forward citation
Original class:
 ['H01L29', 'H01L21'] 
Generated class:
 ['H01L23', 'A61K36']
number
8558345    [H01L27, H01L21, H01L23, H01L29]
8785289                    [H01L21, H01L21]
Name: ipcs, dtype: object
[0.18393972 0.375     ]
[0.36363636 0.33333333]
Generated IPCs are not in 

In [883]:
len(dict_out["index"])

1350

In [884]:
inclusions = np.unique(dict_out["inclusion"], return_counts=True)[1]
ratio_included = np.round(inclusions[1]/inclusions.sum(), 4)
higher_impacts = np.unique(dict_out["higher_impact"], return_counts=True)[1]
ratio_higher_impact = np.round(higher_impacts[1]/higher_impacts.sum(), 4)
print(f"Ratio generated IPCs are included in citing patents: {ratio_included}")
print(f"Ratio generated IPCs have higher impact than other citing patents: {ratio_higher_impact}")

Ratio generated IPCs are included in citing patents: 0.1881
Ratio generated IPCs have higher impact than other citing patents: 0.525


In [886]:
cnt_nonexist

589

In [888]:
cnt_noFC

1192

In [890]:
cnt_diverge

556

In [793]:
dict_out_bck = copy.deepcopy(dict_out) # threshold==0.4

In [794]:
dict_out_bck

{'index': [0,
  3,
  7,
  13,
  14,
  15,
  16,
  23,
  25,
  28,
  29,
  30,
  33,
  37,
  38,
  41,
  43,
  45,
  47,
  53,
  57,
  60,
  62,
  63,
  65,
  66,
  67,
  69,
  72,
  73,
  76,
  77,
  78,
  82,
  83,
  84,
  86,
  89,
  91,
  93,
  94,
  95,
  97,
  99,
  102,
  103,
  106,
  112,
  117,
  120,
  122,
  125,
  126,
  131,
  133,
  134,
  135,
  146,
  147,
  148,
  151,
  153,
  156,
  159,
  160,
  161,
  163,
  164,
  166,
  167,
  169,
  173,
  174,
  180,
  182,
  184,
  187,
  188,
  189,
  192,
  195,
  200,
  201,
  205,
  206,
  210,
  214,
  215,
  218,
  219,
  221,
  223,
  224,
  225,
  234,
  236,
  237,
  238,
  242,
  243,
  244,
  245,
  246,
  249,
  254,
  259,
  262,
  264,
  265,
  267,
  268,
  270,
  275,
  276,
  280,
  281,
  284,
  288,
  289,
  292,
  297,
  304,
  305,
  308,
  311,
  316,
  317,
  321,
  324,
  329,
  330,
  331,
  336,
  340,
  343,
  347,
  348,
  349,
  353,
  354,
  362,
  363,
  364,
  365,
  366,
  368,
  370,
  376,
  